# Langgraph with AgentNeo Integration
 This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with Langgraph, a graph-based approach to managing language models with an agent-based system to enhance the automation and decision-making capabilities of your application. This integration allows for comprehensive analysis and debugging of AI-powered systems.

 # Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [ ]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

In [ ]:

import os
import uuid
from dotenv import load_dotenv
from typing import List, Literal, Annotated
from typing_extensions import TypedDict
import openai

from langchain_core.messages import (
    SystemMessage, 
    HumanMessage, 
    AIMessage, 
    ToolMessage
)
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages


# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI API using environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")



 # Initialize AgentNeo Session and Tracer
 Now, let's set up our AgentNeo session and tracer.



In [ ]:
from agentneo import AgentNeo, Tracer, Execution


# Initialize AgentNeo session
neo_session = AgentNeo(session_name="recipe_builder_assistant")
try:
    neo_session.create_project(project_name="Recipe_Builder")
except:
    neo_session.connect_project(project_name="Recipe_Builder")

# Create tracer
tracer = Tracer(session=neo_session)
tracer.start()


 # Define Agents and Tools
Now, let's create our AI tools using langgraph.

In [ ]:
@tracer.trace_agent(name="RecipeInstructions")
class RecipeInstructions(BaseModel):
    """Instructions for creating a recipe."""
    dish_name: str
    servings: int
    dietary_restrictions: List[str]
    cooking_time: str
    difficulty_level: str
    ingredients: List[str]
    special_equipment: List[str]

@tracer.trace_agent(name="RecipeBuilder")
class RecipeBuilder:
    def __init__(self):
        self.template = """You are a helpful recipe building assistant. Your job is to help users create and customize recipes.

You should gather the following information:
- Name of the dish they want to make
- Number of servings needed
- Any dietary restrictions or preferences
- Desired cooking time
- Cooking skill level
- Available ingredients or preferred ingredients
- Available cooking equipment

If any information is missing or unclear, ask for clarification. Once you have all the necessary information, call the relevant tool to generate the recipe.

Remember to:
- Be specific about ingredient quantities
- Consider dietary restrictions carefully
- Suggest alternatives for uncommon ingredients
- Provide clear, step-by-step instructions"""

        self.llm = ChatOpenAI(temperature=0.7)
        self.llm_with_tool = self.llm.bind_tools([RecipeInstructions])

    @tracer.trace_tool(name="get_messages_info")
    def get_messages_info(self, messages):
        return [SystemMessage(content=self.template)] + messages

    @tracer.trace_tool(name="info_chain")
    def info_chain(self, state):
        messages = self.get_messages_info(state["messages"])
        response = self.llm_with_tool.invoke(messages)
        return {"messages": [response]}

    @tracer.trace_tool(name="get_recipe_messages")
    def get_recipe_messages(self, messages: list):
        tool_call = None
        other_msgs = []
        for m in messages:
            if isinstance(m, AIMessage) and m.tool_calls:
                tool_call = m.tool_calls[0]["args"]
            elif isinstance(m, ToolMessage):
                continue
            elif tool_call is not None:
                other_msgs.append(m)
        
        recipe_system = """Based on the following requirements, create a detailed recipe with:
1. List of ingredients with quantities
2. Required equipment
3. Step-by-step preparation instructions
4. Cooking tips and timing
5. Serving suggestions
6. Storage instructions if applicable

Recipe Requirements:
{reqs}"""
        return [SystemMessage(content=recipe_system.format(reqs=tool_call))] + other_msgs

    @tracer.trace_tool(name="recipe_gen_chain")
    def recipe_gen_chain(self, state):
        messages = self.get_recipe_messages(state["messages"])
        response = self.llm.invoke(messages)
        return {"messages": [response]}

In [ ]:
@tracer.trace_agent(name="NutritionAnalyzer")
class NutritionAnalyzer:
    def __init__(self):
        self.template = """You are a nutrition analysis assistant. Your job is to analyze the nutritional content of recipes.

Given a recipe, you should provide:
1. Total calories per serving
2. Macronutrient breakdown (protein, carbs, fats)
3. Key vitamins and minerals
4. Potential allergens
5. Suggestions for making the recipe healthier (if applicable)

Be as accurate as possible with your estimations. If you're unsure about any information, state that it's an approximation."""

        self.llm = ChatOpenAI(temperature=0.3)

    @tracer.trace_tool(name="get_nutrition_messages")
    def get_nutrition_messages(self, messages):
        recipe = None
        for m in messages:
            if isinstance(m, AIMessage) and not m.tool_calls:
                recipe = m.content
                break
        
        if recipe is None:
            return [SystemMessage(content="No recipe found to analyze.")]
        
        return [
            SystemMessage(content=self.template),
            HumanMessage(content=f"Please analyze the nutritional content of this recipe:\n\n{recipe}")
        ]

    @tracer.trace_tool(name="nutrition_analysis_chain")
    def nutrition_analysis_chain(self, state):
        messages = self.get_nutrition_messages(state["messages"])
        response = self.llm.invoke(messages)
        return {"messages": [response]}

In [ ]:
@tracer.trace_agent(name="CookingTipsAgent")
class CookingTipsAgent:
    def __init__(self):
        self.template = """You are a cooking tips and techniques expert. Your job is to provide helpful advice and tips for the given recipe.

Given a recipe, you should provide:
1. General cooking techniques relevant to the recipe
2. Tips for ingredient preparation
3. Suggestions for enhancing flavors
4. Common mistakes to avoid
5. Time-saving tricks (if applicable)
6. Plating and presentation ideas

Ensure your tips are specific to the recipe and helpful for cooks of all skill levels."""

        self.llm = ChatOpenAI(temperature=0.5)

    @tracer.trace_tool(name="get_cooking_tips_messages")
    def get_cooking_tips_messages(self, messages):
        recipe = None
        for m in messages:
            if isinstance(m, AIMessage) and not m.tool_calls:
                recipe = m.content
                break
        
        if recipe is None:
            return [SystemMessage(content="No recipe found to provide tips for.")]
        
        return [
            SystemMessage(content=self.template),
            HumanMessage(content=f"Please provide cooking tips and techniques for this recipe:\n\n{recipe}")
        ]

    @tracer.trace_tool(name="cooking_tips_chain")
    def cooking_tips_chain(self, state):
        messages = self.get_cooking_tips_messages(state["messages"])
        response = self.llm.invoke(messages)
        return {"messages": [response]}

# Managing state graphs, saving memory, handling messages, and typing annotations.

In [ ]:
# State management

@tracer.trace_tool(name="get_state")
def get_state(state):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "add_tool_message"
    elif isinstance(messages[-1], AIMessage) and not messages[-1].tool_calls:
        if any("nutrition" in m.content.lower() for m in messages[-3:]):
            return "cooking_tips"
        return "nutrition_analysis"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "info"

@tracer.trace_agent(name="State")
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize workflow
memory = MemorySaver()
workflow = StateGraph(State)

# Initialize agents
recipe_builder = RecipeBuilder()
nutrition_analyzer = NutritionAnalyzer()
cooking_tips_agent = CookingTipsAgent()

# Add nodes to workflow
workflow.add_node("info", recipe_builder.info_chain)
workflow.add_node("recipe", recipe_builder.recipe_gen_chain)
workflow.add_node("nutrition_analysis", nutrition_analyzer.nutrition_analysis_chain)
workflow.add_node("cooking_tips", cooking_tips_agent.cooking_tips_chain)

@workflow.add_node
def add_tool_message(state: State):
    return {
        "messages": [
            ToolMessage(
                content="Recipe generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

# Add edges to workflow
workflow.add_conditional_edges("info", get_state, ["add_tool_message", "info", "nutrition_analysis", "cooking_tips", END])
workflow.add_edge("add_tool_message", "recipe")
workflow.add_edge("recipe", "nutrition_analysis")
workflow.add_edge("nutrition_analysis", "cooking_tips")
workflow.add_edge("cooking_tips", END)
workflow.add_edge(START, "info")

# Compile graph
graph = workflow.compile(checkpointer=memory)

# Displaying a PNG image of a graph generated from a LangGraph instance using Mermaid visualization

In [ ]:
# %% 
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Main interaction loop
def run_interaction():
    cached_human_responses = [
        "I want to make chocolate chip cookies",
        "6 servings, no nuts, beginner friendly, 30 minutes cooking time",
        "I have basic baking equipment and ingredients",
        "Can you analyze the nutritional content of this recipe?",
        "Any cooking tips for making these cookies?",
        "q"
    ]
    cached_response_index = 0
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}

    while True:
        try:
            user = input("User (q/Q to quit): ")
        except:
            user = cached_human_responses[cached_response_index]
            cached_response_index += 1
            
        print(f"User (q/Q to quit): {user}")
        
        if user in {"q", "Q"}:
            print("AI: Thank you for using the Recipe Builder, Nutrition Analyzer, and Cooking Tips Assistant! Happy cooking!")
            break
            
        output = None
        for output in graph.stream(
            {"messages": [HumanMessage(content=user)]}, 
            config=config, 
            stream_mode="updates"
        ):
            last_message = next(iter(output.values()))["messages"][-1]
            last_message.pretty_print()

        if output and "cooking_tips" in output:
            print("Recipe, nutrition analysis, and cooking tips completed! Let me know if you need any modifications or have questions!")


# Metrics Evaluation System

In [ ]:

# Metrics Evaluation System
def execute_metrics(neo_session, trace_id):
    """Execute and return metrics analysis."""
    exe = Execution(session=neo_session, trace_id=trace_id)
    exe.execute(metric_list=[
        'goal_decomposition_efficiency',
        'goal_fulfillment_rate',
        'tool_call_success_rate_metric'
    ])
    return exe.get_results()

def print_metrics_results(metric_results):
    """Print the metrics results in a formatted way."""
    print("\nMetrics Results:")
    print(metric_results)

def launch_metrics_dashboard(neo_session):
    """Launch the AgentNeo metrics dashboard."""
    neo_session.launch_dashboard()

In [ ]:
if __name__ == "__main__":
    try:
        run_interaction()
    finally:
        tracer.stop()
        print(f"Trace ID: {tracer.trace_id}")
        


In [ ]:
results = execute_metrics(neo_session, tracer.trace_id)


In [ ]:
# Display results
print_metrics_results(results)
        


In [ ]:
results

In [ ]:
# Launch dashboard
launch_metrics_dashboard(neo_session)